# ALIGNED project: FAST GSA tutorial

**Aligning Life Cycle Assessment methods and bio-based sectors for improved environmental performance**

[http://www.alignedproject.eu/](http://www.alignedproject.eu/)

_Horizon Europe grant agreement N° 101059430. Views and opinions expressed are however those of the author(s) only and do not necessarily reflect those of the European Union or the European Research Executive Agency._ 


## WP1 Shared modelling framework and learnings
### Task 1.4 Framework for interpreting uncertainty

#### Deliverable 1.2 Description of scientific methods

#### Tutorial for performing Global Sensitivity Analysis using the FAST method

Massimo Pizzol, Aalborg University (AAU), 2025  
Ning An, Aalborg University (AAU), 2025

This notebook show how to perform a simple Global Sensitivity Analysis (GSA) applying the FAST, a variance-based sensitivity analysis method, for an example product system of a biobased product.

Credits: special thanks to [Pierre Jouannais](https://www.linkedin.com/in/pierre-jouannais-27736023b/) for preparing an early version of this tutorial.

_Technical note: the SALib library is needed for this tutorial._

In [ ]:
# Importing packages
import bw2data as bd
import bw2calc as bc
import pandas as pd
import numpy as np
from scipy import stats
from lci_to_bw2 import * # import all the functions of this module

In [2]:
# open a project with ecoinvent v.3.11 consequential system model
bd.projects.set_current('advlca25')

We start by importing data about a fictional ("dummy") product system for a biobased product.

In [3]:
# Import the dummy product system

# import data from csv

#mydata = pd.read_csv('LCI1-bw-format.csv', header = 0, sep = ",") # using csv file avoids encoding problem
mydata = pd.read_csv('ALIGNED-LCI-biobased-product-dummy.csv', header = 0, sep = ",") # using csv file avoids encoding problem
mydata.head()

# keep only the columns not needed
mydb = mydata[['Activity database','Activity code','Activity name','Activity unit','Activity type',
               'Exchange database','Exchange input','Exchange amount','Exchange unit','Exchange type',
               'Exchange uncertainty type','Exchange loc','Exchange scale','Exchange negative', 
               'Simapro name',	'Simapro unit', 'Simapro type']].copy()

mydb = mydata.copy()

mydb['Exchange uncertainty type'] = mydb['Exchange uncertainty type'].fillna(0).astype(int) # uncertainty as integers
# Note: to avoid having both nan and values in the uncertainty column I use zero as default

#print(mydb.head())

In [4]:
# Create dictionary in bw format and write database to disk. 
# Shut down all other notebooks using the same project before doing this
bw2_db = lci_to_bw2(mydb) # a function from the lci_to_bw2 module

# write database
bd.Database('ALIGNED-biob-prod-dummy').write(bw2_db)

12:04:27+0200 [warning  ] Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 5/5 [00:00<00:00, 6173.54it/s]

12:04:27+0200 [info     ] Vacuuming database            


The product system includes different activities such as the production, use, and end of life of the biobased product.

In [5]:
# check what foreground activities are included
for act in bd.Database('ALIGNED-biob-prod-dummy'):
    print(act, act['code'])

'Biobased-product-use' (year, None, None) f9eabf64-b899-40c0-9f9f-2009dbb0a0b2
'Biomass-growth' (kilogram, None, None) a7d34649-9c10-4423-bac3-ecab9b43b20c
'Biomass-processing' (kilogram, None, None) 403a5c32-c769-46fc-8b9a-74b8eb3c79d1
'Biobased-product-manufacturing' (kilogram, None, None) a37d149a-6508-4563-8af6-e5a39b4176df
'Biobased-product-eol' (kilogram, None, None) c8301e73-d521-4a89-998b-30b7e7751011


In [6]:
# More info 
myact = bd.Database('ALIGNED-biob-prod-dummy').get('f9eabf64-b899-40c0-9f9f-2009dbb0a0b2') # Biobased-product-use
myact._data

{'name': 'Biobased-product-use',
 'unit': 'year',
 'type': 'processwithreferenceproduct',
 'database': 'ALIGNED-biob-prod-dummy',
 'code': 'f9eabf64-b899-40c0-9f9f-2009dbb0a0b2',
 'id': 228456472572211203}

In [7]:
# Uncertainty is also there
list(myact.exchanges())[1]._data

{'input': ('ALIGNED-biob-prod-dummy', 'a37d149a-6508-4563-8af6-e5a39b4176df'),
 'amount': 50.0,
 'unit': 'kilogram',
 'type': 'technosphere',
 'uncertainty type': 4,
 'minimum': 37.5,
 'maximum': 62.5,
 'Notes': 'Input of manufacturing',
 'Simapro name': 'Biobased-product-manufacturing',
 'Simapro unit': 'kg',
 'output': ('ALIGNED-biob-prod-dummy', 'f9eabf64-b899-40c0-9f9f-2009dbb0a0b2')}

We calculate a static climate impact score for the fictional biobased product, to be used for reference later on.

In [8]:
# calculation of static LCA score
mymethod = ('ecoinvent-3.11', 'IPCC 2021', 'climate change: fossil', 'global warming potential (GWP100)')
myact = bd.Database('ALIGNED-biob-prod-dummy').get('f9eabf64-b899-40c0-9f9f-2009dbb0a0b2') # Biobased-product-use
functional_unit = {myact: 1}
LCA = bc.LCA(functional_unit, mymethod)
LCA.lci()
LCA.lcia()
print(LCA.score)

180.28497364203196


### Now perform sensitivity analysis

The procedure is in three steps.

1) A set of model input parameters is chosen. These are **values of specific exchanges**. A sample of values is produced for each model input **using a specific and efficient sampling design** (the **"problem"** below)
2) A simulation is performed. Initial prameter values are substituted with those in the sample, iteratively, and new model outputs, that are LCA scores, are calculated.
3) **A sensitivity index is calculated** using model input values (the "problem") and output values.

#### Step 1
Obtain a sample of values for each parameter.


This is a  pseudo-random sample obtained using a sampling method from the SALib package on the problem previously defined.

it is defined as a __FAST__ sample ("Fourier Amplitude Sensitivity Analysis"), another alternative to Sobol method.

Sobol and FAST sampling methods use different pseudo-random patterns to cover the space of parameters, and different calculations for the sensitivity indices.

Besides these differences, the (extended) FAST method is, just like the Sobol one, a variance-based method for sensitivity analysis. It provides the same information as the Sobol method but with higher computational efficiency.

For further information and comparison see: 

_Saltelli, A.; Tarantola, S.; Chan, K. P. S. A Quantitative Model-Independent Method for Global Sensitivity Analysis of Model Output. Technometrics 1999, 41 (1), 39–56._ https://doi.org/10.1080/00401706.1999.10485594.


In [9]:
# need to import the SALib library and relaive methods for sensitvity analysis
# do 'pip install SALib' in the brightway environment

from SALib.sample import saltelli
from SALib.sample import fast_sampler
from SALib.analyze import sobol
from SALib.analyze import fast


Define the "problem" for the GSA analysis as indicated in the SALib library.
Uniform distributions for the uncertainty of for each input parameter are here assumed.

In [10]:
problem = { 'num_vars': 4, # number of variables
            'names': ['par1', 'par2', 'par3', 'par4'], # names of variables, same as parameters
            'bounds': [[-1.25, -0.75], # careful here to what is the lower bound with negative values...
                       [0.1, 0.9],  
                       [20, 60],
                       [0.75, 1.25]] ,
           'dists':["unif","unif","unif","unif"] } # all uniform distributions

In [11]:
# FAST sampler creates N*P parameters sets with N=sample size, P = number of parameters
param_values_FAST = fast_sampler.sample(problem, 200)
print(param_values_FAST.shape)

param_values_FAST

(800, 4)


array([[-0.95251116,  0.57598214, 43.7991072 ,  1.04748884],
       [-0.83251116,  0.58398214, 44.5991072 ,  1.06248884],
       [-0.78748884,  0.59198214, 45.3991072 ,  1.07748884],
       ...,
       [-1.10541864,  0.30733018, 29.16650907,  0.95041864],
       [-1.10041864,  0.32333018, 30.36650907,  0.83041864],
       [-1.09541864,  0.33933018, 31.56650907,  0.78958136]])

Associate the values to specific exchanges using the coordinates (column and row) of the technosphere (A) and biosphere (B) matrices, taking the data directly from the BW database that we just created.

In [ ]:
# In biomass growth, value of CO2 uptake
par1 = list(bd.Database('ALIGNED-biob-prod-dummy').get('a7d34649-9c10-4423-bac3-ecab9b43b20c').exchanges())[3]
# In biomass processing, amount of energy used.
par2 = list(bd.Database('ALIGNED-biob-prod-dummy').get('403a5c32-c769-46fc-8b9a-74b8eb3c79d1').exchanges())[1]
# In use of product, amount of manufactured product needed
par3 = list(bd.Database('ALIGNED-biob-prod-dummy').get('f9eabf64-b899-40c0-9f9f-2009dbb0a0b2').exchanges())[1]
# In use of product, amount of manufactured product needed
par4 = list(bd.Database('ALIGNED-biob-prod-dummy').get('c8301e73-d521-4a89-998b-30b7e7751011').exchanges())[2]

n_iter = len(param_values_FAST)
param_samples = [(par1['output'],par1['input'], [i[0] for i in param_values_FAST]),
                 (par2['output'],par2['input'], [i[1] for i in param_values_FAST]),
                 (par3['output'],par3['input'], [i[2] for i in param_values_FAST]),
                 (par4['output'],par4['input'], [i[3] for i in param_values_FAST])]

#### Step 2

Iteration through all parameter samples. New LCA scores are calculated for each combination of values.

With 800 iterations, **this will take some time**

In [13]:
# Implementing the loop

GSA_value_results = []


for i in range(0,len(param_values_FAST)): 
    for s in param_samples:

        # Get the 'id' using the activity object
        col_id = bd.Database(s[0][0]).get(s[0][1]).id
        row_id = bd.Database(s[1][0]).get(s[1][1]).id

        if s[1][0] == "ecoinvent-3.11-biosphere":
            col = LCA.activity_dict[col_id] # find column index of A matrix for the activity
            row = LCA.biosphere_dict[row_id] # find row index of B matrix for the exchange
            LCA.biosphere_matrix[row,col] = s[2][i] # substitute the value
        
        else:
            col = LCA.activity_dict[col_id] # find column index of A matrix for the activity
            row = LCA.activity_dict[row_id] # find row index of B matrix for the exchange
            LCA.technosphere_matrix[row,col] = -s[2][i] # substitute the value
        
    LCA.redo_lci() # uses the new A matrix
    LCA.lcia()
    #print(LCA.score)
    GSA_value_results.append(LCA.score)

#### Step 3

Now we have both the input and output values and we can feed the problem and the results to the Sobol function to obtain the FAST indices.

First we look at the parameters and the result



In [14]:
# Organize data first to give a look at what we have done: a sample of input values and a corresponding output value
fast_data = pd.DataFrame([i[2] for i in param_samples], index = ['par0','par1', 'par2', 'par3']).T
fast_data['GWI'] = GSA_value_results
fast_data.head(10) # show only the first ten samples

,par0,par1,par2,par3,GWI
0,-0.952511,0.575982,43.799107,1.047489,167.723874
1,-0.832511,0.583982,44.599107,1.062489,173.245951
2,-0.787489,0.591982,45.399107,1.077489,177.162311
3,-0.907489,0.599982,46.199107,1.092489,177.302990
4,-1.027489,0.607982,46.999107,1.107489,177.347911
5,-1.147489,0.615982,47.799107,1.122489,177.297076
6,-1.232511,0.623982,48.599107,1.137489,178.000426
7,-1.112511,0.631982,49.399107,1.152489,183.699960
8,-0.992511,0.639982,50.199107,1.167489,189.495738
9,-0.872511,0.647982,50.999107,1.182489,195.387758


Now we calculate the SI index

In [15]:
si = fast.analyze(problem, np.array(GSA_value_results), print_to_console=True) # must use np.array

            S1        ST   S1_conf   ST_conf
par1  0.008731  0.010354  0.069247  0.091341
par2  0.000032  0.000809  0.066464  0.088662
par3  0.934124  0.937016  0.062582  0.097299
par4  0.055131  0.057432  0.084735  0.089503


In this specific case "par3"  is the parameter to which the results are most sensitive to, this because it has the highest value for "S1" which is the first order effect.

This is determined with relatively good confidence as the error (value of "S1_conf") is small compared to the coefficient.

Note also that "par3" is also the parameter with highest sensitivity in the total model, i.e. when in combinatino with other parameters, as observed by the highest value of "ST".

In [16]:
# In this specific case "par3" is the parameter to which the results ar emost sensitive to, and values have high confidente (small error)
# what was "par3" again?
par3 # amount of manufactured product need din the use stage

Exchange: 50.0 kilogram 'Biobased-product-manufacturing' (kilogram, None, None) to 'Biobased-product-use' (year, None, None)>